# Advent of Code 2024
Solutions are in order based on when I solved them.

# Day 4

In [72]:
directions = [
    [-1, 0],  # left (0)
    [-1, -1],  # up-left (1)
    [0, -1],  # up (2)
    [1, -1],  # up-right (3)
    [1, 0],  # right (4)
    [1, 1],  # down-right (5)
    [0, 1],  # down (6)
    [-1, 1],  # down-left (7)
]

def get_neighbor_coordinates(grid, direction, x, y):
    transformations = directions[direction]
    newX = x + transformations[0]
    newY = y + transformations[1]
    return newX, newY


def safe_get_value(grid, x, y):
    if x < 0 or y < 0 or x >= len(grid[0]) or y >= len(grid):
        return None
    return grid[y][x]


def get_neighbor(grid, direction, x, y):
    newX, newY = get_neighbor_coordinates(grid, direction, x, y)
    return safe_get_value(grid, newX, newY)


def get_matching_neighbor_directions(grid, x, y):
    matching_directions = []
    for i in range(len(directions)):
        if get_neighbor(grid, i, x, y) == "M":
            matching_directions.append(i)
    return matching_directions


def try_path(grid, x, y, direction):
    # should be M
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, x, y)
    # want to find A
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, curr_x, curr_y)
    cursor = safe_get_value(grid, curr_x, curr_y)
    if (cursor != "A"):
        return False
    # want to find S
    curr_x, curr_y = get_neighbor_coordinates(grid, direction, curr_x, curr_y)
    cursor = safe_get_value(grid, curr_x, curr_y)
    if (cursor != "S"):
        return False    
    return True
        

def process_puzzle(puzzle):
    rows = puzzle.split("\n")
    grid = [list(row) for row in rows]
    return grid


def runner():
    with open("./inputs/day4.txt") as f:
        day4 = f.read()
    grid = process_puzzle(day4)
    num = 0
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            cursor = grid[y][x]
            if cursor == "X":
                matching_neighbor_directions = get_matching_neighbor_directions(grid, x, y)
                for dir in matching_neighbor_directions:
                    if try_path(grid, x, y, dir) == True:
                        num += 1
    return num


def is_unique_match(corner1, corner2):
    if (corner1 != corner2) and (corner1 == "M" or corner1 == "S") and (corner2 == "M" or corner2 == "S"):
        return True
    return False


def check_diagonals(grid, x, y):
    ul = get_neighbor(grid, 1, x, y)
    dr = get_neighbor(grid, 5, x, y)
    if is_unique_match(ul, dr) == False:
        return False
    
    ur = get_neighbor(grid, 3, x, y)
    dl = get_neighbor(grid, 7, x, y)
    if is_unique_match(ur, dl) == False:
        return False
    
    return True

def runner_pt2():
    with open("./inputs/day4.txt") as f:
        day4 = f.read()
    grid = process_puzzle(day4)
    num = 0
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            cursor = grid[y][x]
            if cursor == "A":
                if check_diagonals(grid, x, y) == True:
                    num += 1
    return num

print(runner())
print(runner_pt2())

2554
1916


# Day 5

In [73]:
def read_inputs():
    with open("./inputs/day5-rules.txt") as f:
        day5_rules = f.read().splitlines()
    with open("./inputs/day5-pages.txt") as f:
        day5_pages = f.read().splitlines()
    return day5_rules, day5_pages


def process_inputs(rules, pages):
    # construct rule map where AFTER constraint is key and all BEFORE constraints are values
    # ex. {5: [1, 2, 3, 4]} indicates that 5 must come after 1, 2, 3, and 4
    rule_map = {}
    for rule in rules:
        [before, after] = [int(x) for x in rule.split("|")]
        if after in rule_map:
            rule_map[after] = rule_map[after] + [before]
        else:
            rule_map[after] = [before]
    # construct candidates as a 2D array of ints
    candidates = [[int(s) for s in x.split(",")] for x in pages]
    return rule_map, candidates


def check_candidate(rule_map, candidate):
    disallow_set = set()
    for num in candidate:
        if num in disallow_set:
            return False
        if num in rule_map:
            disallow_set.update(rule_map[num])
    return True


def fix_candidate(rule_map, candidate):
    fixed_candidate = [-1] * len(candidate)
    for num in candidate:
        if num in rule_map:
            applicable_rules = list(filter(lambda x: x in candidate, rule_map[num]))
            fixed_candidate[len(applicable_rules)] = num
    return fixed_candidate


def get_middle_value(arr):
    return arr[len(arr) // 2]


def runner():
    valid_sum = 0
    invalid_sum = 0
    rules, pages = read_inputs()
    rule_map, candidates = process_inputs(rules, pages)
    for i, candidate in enumerate(candidates):
        # part 1
        if check_candidate(rule_map, candidate):
            valid_sum += get_middle_value(candidate)
        # part 2
        else:
            fixed_candidate = fix_candidate(rule_map, candidate)
            invalid_sum += get_middle_value(fixed_candidate)
    return valid_sum, invalid_sum


runner()

(5087, 4971)

# Day 6

In [74]:
import time
from IPython.display import clear_output, display, HTML
import html

directions = ["^", ">", "v", "<"]
directionMap = {"^": [0, -1], ">": [1, 0], "v": [0, 1], "<": [-1, 0]}


def read_inputs():
    with open("./inputs/day6.txt") as f:
        arr = f.read().splitlines()
    grid = [list(row) for row in arr]
    return grid


def rotate(direction):
    i = directions.index(direction)
    newI = (i + 1) % len(directions)
    return directions[newI]


def stringifyCoors(x, y, dir):
    return f"{x}|{y}", f"{x}|{y}|{dir}"


def pretty_print(grid):
    gridTxt = "\n".join(["".join(row) for row in grid])
    msg = html.escape(gridTxt)
    clear_output(wait=True)
    display(
        HTML(
            f"<pre style='font-family: monospace; font-size: 12px; line-height: 1'>{msg}</pre>"
        )
    )
    time.sleep(0.005)


def find_cursor(grid):
    for y, row in enumerate(grid):
        for x, item in enumerate(row):
            if item in directions:
                return [x, y]
    return None


def try_step(grid, x, y, direction):
    # pretty print for debugging and fun! comment it out for time efficiency.
    # pretty_print(grid)
    # calculate new spot
    transformation = directionMap[direction]
    newX = x + transformation[0]
    newY = y + transformation[1]

    # exited the grid
    if newX < 0 or newX >= len(grid[0]) or newY < 0 or newY >= len(grid):
        return False

    # hit a barrier
    if grid[newY][newX] == "#":
        newDir = rotate(direction)
        return x, y, newDir, False

    # perform step forward
    return newX, newY, direction, True


def compute_path(grid, startX, startY, startDir):
    cursorX, cursorY, direction = startX, startY, startDir
    coors, coors_with_dir = stringifyCoors(cursorX, cursorY, direction)
    visited = {coors}
    visited_with_dir = {coors_with_dir}
    path = [(cursorX, cursorY, direction)]

    while True:
        step = try_step(grid, cursorX, cursorY, direction)
        if step == False:
            break
        cursorX, cursorY, newDir, moved = step
        if not moved:
            direction = newDir
            continue
        direction = newDir
        coors, coors_with_dir = stringifyCoors(cursorX, cursorY, direction)
        # check for loops
        if coors_with_dir in visited_with_dir:
            return True, visited, path
        visited.add(coors)
        visited_with_dir.add(coors_with_dir)
        path.append((cursorX, cursorY, direction))
    return False, visited, path


def count_loops(grid, path):
    startX, startY, startDir = path[0]
    loops = set()
    for step in path[1:]:
        currX, currY, _ = step
        grid[currY][currX] = "#"
        isLoop, _, _ = compute_path(grid, startX, startY, startDir)
        grid[currY][currX] = "."
        if isLoop == True:
            coors, _ = stringifyCoors(currX, currY, "X")
            loops.add(coors)
    return loops


def runner():
    grid = read_inputs()
    [cursorX, cursorY] = find_cursor(grid)
    direction = grid[cursorY][cursorX]

    _, visitedSet, path = compute_path(grid, cursorX, cursorY, direction)
    loops = count_loops(grid, path)

    return len(visitedSet), len(loops)


runner()

(5531, 2165)

# Day 9

In [75]:
def process_input():
    with open("./inputs/day9.txt") as file:
        fileStr = file.read()
    input = [int(char) for char in list(fileStr)]
    return input


def arrange(arr):
    disk = []
    file_space = []
    free_space = []
    for i, item in enumerate(arr):
        if i % 2 == 0:
            file_space.append({"index": len(disk), "space": item})
            disk.extend([i // 2] * item)
        else:
            if item > 0:
                free_space.append({"index": len(disk), "space": item})
            disk.extend(["."] * item)
    return disk, free_space, file_space


def fragment_part1(disk):
    left = 0
    right = len(disk) - 1
    target = None
    while left < right:
        if target != None:
            if disk[left] == ".":
                disk[left] = target
                disk[right] = "."
                target = None
            else:
                left += 1
            continue
        if disk[right] == ".":
            right -= 1
            continue
        else:
            target = disk[right]
            continue
    return disk


def fragment_part2(disk, free_space, file_space):
    for file in reversed(file_space):
        remove_target = None
        for index, free in enumerate(free_space):
            if free['index'] >= file['index']:
                break
            if free['space'] >= file['space']:
                target = disk[file['index']]
                for i in range(file['space']):
                    disk[free['index'] + i] = target
                    disk[file['index'] + i] = "."
                free['space'] -= file['space']
                free['index'] += file['space']
                if free['space'] <= 0:
                    remove_target = index
                break
        # This results in a significant speed up!
        if remove_target != None:
            free_space.pop(remove_target)
    return disk


def checksum(disk):
    sum = 0
    for i, item in enumerate(disk):
        if item != ".":
            sum += i * item
    return sum


def runner():
    # input = [int(char) for char in list("2333133121414131402")]
    input = process_input()
    disk, free_space, file_space = arrange(input)
    disk_two = disk.copy()

    fragment_part1(disk)
    print(checksum(disk))

    fragment_part2(disk_two, free_space, file_space)
    print(checksum(disk_two))
    
    # print("".join([str(x) for x in disk_two]))


runner()

6332189866718
6353648390778


# Day 10

In [76]:
def process_input():
    with open("./inputs/day10.txt") as file:
        arr = file.read().splitlines()
    return [[int(c) for c in list(row)] for row in arr]


def find_trailheads(grid):
    coors = []
    for y, row in enumerate(grid):
        for x, item in enumerate(row):
            if item == 0:
                coors.append((x, y))
    return coors


def get_neighbors(grid, x, y):
    transformations = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    item = grid[y][x]
    neighbors = []
    for t in transformations:
        dx, dy = t
        newX = x + dx
        if newX < 0 or newX >= len(grid[0]):
            continue
        newY = y + dy
        if newY < 0 or newY >= len(grid):
            continue
        if item + 1 == grid[newY][newX]:
            neighbors.append((newX, newY))
    return neighbors


def dfs(grid, originX, originY, target):
    search_stack = get_neighbors(grid, originX, originY)
    paths = []
    targets = set()
    path = [(originX, originY)]
    while len(search_stack) > 0:
        neighbor = search_stack.pop()
        x, y = neighbor
        path.append(neighbor)
        if grid[y][x] == target:
            paths.append(path.copy())
            targets.add(neighbor)
            path.pop()
        else:
            new_neighbors = get_neighbors(grid, x, y)
            if len(new_neighbors) == 0:
                path.pop()
            else:
                search_stack.extend(new_neighbors)
    return paths, targets


def visualize(grid, path):
    print("Length", len(path))
    modified_grid = [[str(item) if (x, y) in path else "." for x, item in enumerate(row)] for y, row in enumerate(grid)]
    for row in modified_grid:
        print("".join(row))


def runner():
    grid = process_input()
    trailheads = find_trailheads(grid)
    part1_score = 0
    part2_rating = 0
    for t in trailheads:
        x, y = t
        paths, targets = dfs(grid, x, y, 9)
        part1_score += len(targets)
        part2_rating += len(paths)
    return part1_score, part2_rating
    

runner()

(496, 1120)

# Day 7

In [77]:
def process_input():
    with open("./inputs/day7.txt") as file:
        arr = file.read().splitlines()
        processes = []
        for line in arr:
            [answer, operands] = line.split(": ")
            processes.append({"answer": int(answer), "operands": [int(n) for n in operands.split(" ")]})
    return processes


def generate_permutations(elements, length):
    stack = [[]]
    results = []

    while stack:
        current = stack.pop()
        if len(current) == length:
            results.append(current)
        else:
            for element in elements:
                stack.append(current + [element])

    return results


def evaluate(operator, a, b):
    if operator == "+":
        return a+b
    elif operator == "*":
        return a*b
    # disable this for part 1
    elif operator == "||":
        return int(f"{a}{b}")
    return -1


def test_operators(experiment):
    num_operators = len(experiment['operands']) - 1
    
    possible_operators = generate_permutations(['+', '*', '||'], num_operators)
    for operators in possible_operators:
        solution = None
        for i, o in enumerate(operators):
            if solution == None:
                solution = evaluate(o, experiment['operands'][0], experiment['operands'][1])
            else:
                solution = evaluate(o, solution, experiment['operands'][i+1])
            if solution > experiment['answer']:
                break
        if solution == experiment['answer']:
            return experiment['answer'], operators
    return False


def runner():
    experiments = process_input()
    sum = 0
    for e in experiments:
        result = test_operators(e)
        if result != False:
            answer, operators = result
            sum += answer
    print(sum)


runner()

34612812972206


# Day 8

In [78]:
def process_input():
    with open("./inputs/day8.txt") as file:
        arr = file.read().splitlines()
        grid = [list(row) for row in arr]
    return grid


def find_antennas(grid):
    map = {}
    for y, row in enumerate(grid):
        for x, item in enumerate(row):
            if item != ".":
                if item in map:
                    map[item] += [(x, y)]
                else:
                    map[item] = [(x, y)]
    return map


def create_pairs(arr):
    pairs = []
    for i, item in enumerate(arr):
        for j in range(i + 1, len(arr)):
            pairs.append((item, arr[j]))
    return pairs


def check_in_bounds(grid, coors):
    x, y = coors
    if x < 0 or y < 0 or x >= len(grid[0]) or y >= len(grid):
        return False
    return True


def diff(coors1, coors2):
    x1, y1 = coors1
    x2, y2 = coors2
    dx = x2 - x1
    dy = y2 - y1
    return (dx, dy)


def translate(coors, d, dir):
    x, y = coors
    dx, dy = d
    return (x + dx, y + dy) if dir > 0 else (x - dx, y - dy)


def calculate_antinodes_part1(grid, pair):
    coors1, coors2 = pair
    d = diff(coors1, coors2)
    ant1 = translate(coors1, d, -1)
    ant2 = translate(coors2, d, 1)
    antinodes = []
    if check_in_bounds(grid, ant1):
        antinodes.append(ant1)
    if check_in_bounds(grid, ant2):
        antinodes.append(ant2)
    return antinodes


def calculate_antinodes_part2(grid, pair):
    coors1, coors2 = pair
    d = diff(coors1, coors2)
    antinodes = [coors1, coors2]
    ant_neg = coors1
    ant_pos = coors2
    while True:
        ant_neg = translate(ant_neg, d, -1)
        if check_in_bounds(grid, ant_neg):
            antinodes.append(ant_neg)
        else:
            break
    while True:
        ant_pos = translate(ant_pos, d, 1)
        if check_in_bounds(grid, ant_pos):
            antinodes.append(ant_pos)
        else:
            break
    return antinodes


def runner():
    grid = process_input()
    map = find_antennas(grid)
    antinodes_part1 = set()
    antinodes_part2 = set()
    for key in map.keys():
        pairs = create_pairs(map[key])
        for pair in pairs:
            antinodes_part1.update(calculate_antinodes_part1(grid, pair))
    for key in map.keys():
        pairs = create_pairs(map[key])
        for pair in pairs:
            antinodes_part2.update(calculate_antinodes_part2(grid, pair))

    return len(antinodes_part1), len(antinodes_part2)


runner()

(329, 1190)

# Day 11

In [79]:
def process_input():
    input = "4610211 4 0 59 3907 201586 929 33750"
    arr = [int(num) for num in input.split(" ")]
    state = {}
    for item in arr:
        if item in state:
            state[item] += 1
        else:
            state[item] = 1
    return state


# Worked fine for part 1, way too slow for part 2
def blink_v1(state):
    offset = 0
    for i, item in enumerate(state.copy()):
        if int(item) == 0:
            state[i + offset] = "1"
        elif len(item) % 2 == 0:
            half = len(item) // 2
            a = item[:half]
            b = str(int(item[half:]))
            state[i + offset] = a
            offset += 1
            state.insert(i + offset, b)
        else:
            state[i + offset] = str(int(state[i + offset]) * 2024)


def int_len(num):
    target = num
    digits = 0
    while target > 0:
        target = target // 10
        digits += 1
    return digits


def half(num, digits):
    return num // 10 ** (digits // 2), num % 10 ** (digits // 2)


# Optimized, but still never finishes for part 2
def blink_v2(state):
    insertions = []
    for i, item in enumerate(state):
        if item == 0:
            state[i] = 1
            continue
        l = int_len(item)
        if l % 2 == 0:
            a, b = half(item, l)
            state[i] = a
            insertions.append(b)
        else:
            state[i] *= 2024
    state.extend(insertions)
    

def increment(state, key, multiplier):
    if key in state:
        state[key] += multiplier
    else:
        state[key] = multiplier


# Optimized with memoization
def blink_v3(state, memo):
    new_state = {}
    for item in list(state.keys()):
        multiplier = state.pop(item)
        if item in memo:
            for child in memo[item]:
                increment(new_state, child, multiplier)
        else:
            l = int_len(item)
            if l % 2 == 0:
                a, b = half(item, l)
                memo[item] = [a, b]
                increment(new_state, a, multiplier)
                increment(new_state, b, multiplier)
            else:
                p = item * 2024
                memo[item] = [p]
                increment(new_state, p, multiplier)
    return new_state


def runner():
    # change for part 1
    l = 75 # 25
    state = process_input()
    memo = {0: [1]}
    for i in range(l):
        state = blink_v3(state, memo)
    return sum(state.values())


runner()

234568186890978

# Day 1

In [92]:
def process_input():
    left = []
    right = []
    with open("./inputs/day1.txt") as file:
        rows = file.read().splitlines()
    for r in rows:
        [l, r] = r.split("  ")
        left.append(int(l))
        right.append(int(r))
    return left, right


def compute_distance(left, right):
    d = 0
    for i in range(len(left)):
        d += abs(left[i]-right[i])
    return d


def compute_similarity(left, right):
    freq = {}
    for item in right:
        if item in freq:
            freq[item] += 1
        else:
            freq[item] = 1
    s = 0
    for l in set(left):
        multiplier = 0 if l not in freq else freq[l]
        s += (l * multiplier)
    return s


def runner():
    left, right = process_input()
    left.sort()
    right.sort()
    d = compute_distance(left, right)
    s = compute_similarity(left, right)
    return d, s


runner()

(936063, 23150395)

# Day 2

In [156]:
def process_input():
    with open("./inputs/day2.txt") as file:
        arr2d = [[int(num) for num in row.split(" ")] for row in file.read().splitlines()]
    return arr2d


def check_report(report):
    dir = 0
    prev = report[0]
    for level in report[1:]:
        diff = level - prev
        prev = level
        if diff == 0 or abs(diff) > 3 or (diff > 0 and dir < 0) or (diff < 0 and dir > 0):
            return False
        if dir == 0:
            dir = diff
    return True


def check_removal(report):
    for i in range(len(report)):
        tmp = report.copy()
        tmp.pop(i)
        if check_report(tmp):
            return True
    return False


def runner():
    reports = process_input()
    valid_part1 = 0
    valid_part2 = 0
    for report in reports:
        if check_report(report):
            valid_part1 += 1
            valid_part2 += 1
        elif check_removal(report):
            valid_part2 += 1
    return valid_part1, valid_part2


runner()

(432, 488)

# Day 12

*My apologies to anyone reading day 12 code; it is quite messy.*

In [247]:
def process_input():
    with open("./inputs/day12.txt") as file:
        grid = [list(row) for row in file.read().splitlines()]
    return grid


def get_matching_neighbors(grid, x, y, target):
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    neighbors = []
    edges = []
    for d in directions:
        dx, dy = d
        newX = x + dx
        newY = y + dy
        if (newX >= 0 and newY >= 0 and newX < len(grid[0]) and newY < len(grid) and grid[newY][newX] == target):
            neighbors.append((newX, newY))
        else:
            edges.append(d)
    return neighbors, edges

def dfs(grid, originX, originY):
    target = grid[originY][originX]
    search_stack, origin_edges = get_matching_neighbors(grid, originX, originY, target)
    visited = {(originX, originY)}
    perimeter = 4-len(search_stack)
    edge_map = {(1, 0): set(), (0, 1): set(), (-1, 0): set(), (0, -1): set()}
    for edge in origin_edges:
        edge_map[edge].add((originX, originY))
    while len(search_stack) > 0:
        x, y = search_stack.pop()
        if (x, y) in visited:
            continue
        visited.add((x, y))
        target = grid[y][x]
        neighbors, edges = get_matching_neighbors(grid, x, y, target)
        perimeter += (4-len(neighbors))
        for edge in edges:
            edge_map[edge].add((x, y))
        unvisited_neighbors = list(filter(lambda neighbor: neighbor not in visited, neighbors))
        search_stack.extend(unvisited_neighbors)
    return target, visited, perimeter, edge_map


def compute_sides(edges):
    disconnected_sides = []
    sides = 0
    for direction in edges.keys():
        axis_map = {}
        axis_selector = 0 if direction[0] != 0 else 1
        point_selector = 0 if direction[0] == 0 else 1
        for edge in edges[direction]:
            axis = edge[axis_selector]
            point = edge[point_selector]
            if axis in axis_map:
                axis_map[axis].add(point)
            else:
                axis_map[axis] = {point}
        disconnected_sides.append([sorted(list(point_set)) for point_set in axis_map.values()])
    for a in disconnected_sides:
        for b in a:
            if len(b) == 0:
                continue
            if len(b) == 1:
                sides += 1
                continue
            sides += 1
            prev = b[0]
            for c in b[1:]:
                if c-prev > 1:
                    sides += 1
                prev = c
    return sides
        

def runner():
    grid = process_input()
    unvisited = set([(x, y) for x in range(len(grid[0])) for y in range(len(grid))])
    
    cost = 0
    discount_cost = 0
    while len(unvisited) > 0:
        x, y = unvisited.pop()
        target, visited, perimeter, edges = dfs(grid, x, y)
        sides = compute_sides(edges)
        cost += (perimeter * len(visited))
        discount_cost += (sides * len(visited))
        unvisited.difference_update(visited)
    return cost, discount_cost
    
runner()

(1402544, 862486)

# Day 13